In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [ff71e718] MixedModels v4.4.0 `~/.julia/dev/MixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
using NLopt
using HighDimMixedModels

In [3]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


#### parse random effect in formula

In [4]:
f = @formula(period ~ 0 + incid + hsz + (1|herd))
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [9]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = -57.1099018030012
The initial object value is -57.1099018030012
sigma = [1.0, 1.0]
negLog = -57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -57.1099018030012
sigma = [1.75, 1.0]
negLog = -55.97912076973617
sigma = [1.0, 1.75]
negLog = -52.37720093978967
sigma = [0.25, 1.0]
negLog = -64.50473446075652
sigma = [1.0, 0.25]
negLog = -128.22481961167495
sigma = [0.29289321881345254, 0.0]
negLog = Inf
sigma = [0.9469669914110089, 0.19696699141100893]
negLog = -156.76581599673838
sigma = [0.995676667220638, 0.13993745974082217]
negLog = -207.31013755591323
sigma = [0.9952996645647639, 0.06493840728682465]
negLog = -405.77399467791014
sigma = [0.9878315127277438, 0.06424796674686986]
negLog = -410.13655848976134
sigma = [0.987341243774038, 0.05676400814509819]
negLog = -458.32682425399554
sigma = [0.9948025184365511, 0.05600283772361585]
negLog = -463.4181543729141
sigma = [0.9947292842170775, 0.05525642178789495]
negLog = -469.0572593

## debug opt

In [50]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
#C = randn((n-r),n)
C = transpose(u[:,1:(n-r)]) 
K = C*P
Z = HMM.Z.Z
y = HMM.y
# C can be any full rank matrix with size n,r, e.g. randn(n,r)
print()

In [51]:
K

54×56 Matrix{Float64}:
 -1.1638e-17    0.164752    -0.0399541   …   0.0674121    -0.222677
  1.8836e-18   -0.403341     0.123302       -0.241088     -0.152237
  2.81172e-18   0.0346548    0.212919       -0.0523286     0.0707382
 -2.59675e-18   0.0221316    0.0389036       0.34114       0.148062
 -1.5352e-18   -0.0448844   -0.0968939       0.237301     -0.0157088
 -3.10139e-18  -0.00807415   0.0250054   …  -0.0915462     0.139485
  6.74461e-18   0.0739587    0.135703       -0.171519     -0.0101518
  3.12195e-18  -0.0232285   -0.101904       -0.0211855     0.36347
  9.47794e-18  -0.0120339    0.0262741       0.168954      0.0790729
  1.23066e-17   0.00766534   0.183135        0.124075     -0.064429
 -7.64651e-19  -0.0546021   -0.0707007   …  -0.0410829    -0.0921613
  2.94435e-18   0.134913     0.191127       -0.100892     -0.00686268
 -6.46766e-18   0.097094     0.190483        0.162945     -0.106228
  ⋮                                      ⋱                 ⋮
 -1.27162e-18   0.093824  

In [37]:
A

56×2 Matrix{Float64}:
  2.0  14.0
  3.0  12.0
  4.0   9.0
  0.0   5.0
  3.0  22.0
  1.0  18.0
  1.0  21.0
  8.0  22.0
  2.0  16.0
  0.0  16.0
  2.0  20.0
  2.0  10.0
  0.0  10.0
  ⋮    
  1.0  21.0
  2.0  24.0
  0.0  19.0
  0.0  23.0
 11.0  19.0
  0.0   2.0
  0.0   3.0
  0.0   2.0
  1.0  19.0
  1.0  15.0
  1.0  15.0
  0.0  15.0

#### (a) construct Opt by optsum object

In [12]:
alg = :LN_COBYLA
verbose = true
## add optsum
# init para
sigma = [1.0,1.0]
lbd = [0.0; 0.0]
optsum = OptSummary(Float64.(sigma), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
opt = Opt(optsum)

function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(sigma), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0])  # the second field not useful right now

sigma = [1.0, 1.0]
negLog = -57.1099018030012
The initial object value is -57.1099018030012
sigma = [1.0, 1.0]
negLog = -57.1099018030012


-57.1099018030012

In [13]:
if verbose println("OPTBL: starting point $(sigma)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, sigma)

OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -57.1099018030012
sigma = [1.75, 1.0]
negLog = -55.97912076973617
sigma = [1.0, 1.75]
negLog = -52.37720093978967
sigma = [0.8257088595036971, 0.27053266122156105]
negLog = -123.76838880255191
sigma = [0.12620040407776012, 0.0]
negLog = Inf
sigma = [NaN, NaN]
negLog = NaN
sigma = [NaN, NaN]
negLog = NaN
sigma = [0.829124063101169, 0.2515771049569857]
negLog = -131.0300978457053
sigma = [1.0049609484577022, 0.1864799487355855]
negLog = -162.54073371434723
sigma = [1.0248266656543088, 3.5310088440543286e-5]
negLog = -648060.2927720573
sigma = [0.8373266689791151, 8.328705563762084e-17]
negLog = Inf
sigma = [NaN, NaN]
negLog = NaN
sigma = [NaN, NaN]
negLog = NaN
sigma = [1.0248237372088926, 0.01318516660603881]
negLog = -1786.9679007265013
sigma = [NaN, NaN]
negLog = NaN
sigma = [1.042117729926439, 3.5716815641248804e-5]
negLog = -640677.9057947188
sigma = [0.9779516789535357, 0.0]
negLog = Inf
sigma = [NaN, NaN]
negLog = NaN
si

(-1.968469929933476e6, [1.0248268796675095, 1.1625301820233277e-5], :XTOL_REACHED)

In [52]:
Sigma = optx[1]*Z*transpose(Z) + optx[2]*diagm(ones(n))
beta = inv(transpose(A)*inv(Sigma)*A)*transpose(A)*inv(Sigma)*y
betaM = beta[1:size(HMM.M,2)]
betaX = beta[(size(HMM.M,2) + 1): size(beta)[1]]

optsum.feval = opt.numevals
optsum.final = optx
optsum.fmin = optf
optsum.returnvalue = ret

if verbose println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $(opt.numevals) iterations (returned $(ret))") ; end

HMM.optsum = optsum

got -1.96846992993e6 at [1.02483, 1.0e-5] after 61 iterations (returned XTOL_REACHED)


|                          |                    |
|:------------------------ |:------------------ |
| **Initialization**       |                    |
| Initial parameter vector | [1.0, 1.0]         |
| Initial objective value  | -57.1099018030012  |
| **Optimizer settings**   |                    |
| Optimizer (from NLopt)   | `LN_COBYLA`        |
| Lower bounds             | [0.0, 0.0]         |
| `ftol_rel`               | 1.0e-12            |
| `ftol_abs`               | 1.0e-8             |
| `xtol_rel`               | 1.0e-5             |
| `xtol_abs`               | [1.0e-10, 1.0e-10] |
| `initial_step`           | [0.75, 0.75]       |
| `maxfeval`               | -1                 |
| `maxtime`                | -1.0               |
| **Result**               |                    |
| Function evaluations     | 61                 |
| Final parameter vector   | [1.0248, 0.0]      |
| Final objective value    | -1.9684699299e6    |
| Return code              | `XTOL_REACHED`     |


#### (b) construct opt by Opt(:LN_COBYLA, 2)

In [32]:
using NLopt
alg = :LN_COBYLA
verbose = true
## add optsum
# init para
sigma = [1.0,1.0]
lbd = [0.0; 0.0]
optsum = OptSummary(Float64.(sigma), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
#opt = Opt(optsum)

## init opt based on Opt function
opt = Opt(:LN_COBYLA, 2)

NLopt.ftol_rel!(opt, optsum.ftol_rel) # relative criterion on objective
NLopt.ftol_abs!(opt, optsum.ftol_abs) # absolute criterion on objective
NLopt.xtol_rel!(opt, optsum.xtol_rel) # relative criterion on parameter values
NLopt.xtol_abs!(opt, optsum.xtol_abs) # absolute criterion on parameter values
NLopt.lower_bounds!(opt, lbd)
NLopt.maxeval!(opt, optsum.maxfeval)
NLopt.maxtime!(opt, optsum.maxtime)
optsum.initial_step = NLopt.initial_step(opt, optsum.initial, similar(lbd))

2-element Vector{Float64}:
 0.75
 0.75

In [33]:
opt

Opt(LN_COBYLA, 2)

In [34]:
function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(sigma), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0])  # the second field not useful right now

if verbose println("OPTBL: starting point $(sigma)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, sigma)

sigma = [1.0, 1.0]
negLog = -57.1099018030012
The initial object value is -57.1099018030012
sigma = [1.0, 1.0]
negLog = -57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -57.1099018030012
sigma = [1.75, 1.0]
negLog = -55.97912076973617
sigma = [1.0, 1.75]
negLog = -52.37720093978967
sigma = [0.8257088595036971, 0.27053266122156105]
negLog = -123.76838880255191
sigma = [0.12620040407776012, 0.0]
negLog = Inf
sigma = [NaN, NaN]
negLog = NaN
sigma = [NaN, NaN]
negLog = NaN
sigma = [0.829124063101169, 0.2515771049569857]
negLog = -131.0300978457053
sigma = [1.0049609484577022, 0.1864799487355855]
negLog = -162.54073371434723
sigma = [1.0248266656543088, 3.5310088440543286e-5]
negLog = -648060.2927720573
sigma = [0.8373266689791151, 8.328705563762084e-17]
negLog = Inf
sigma = [NaN, NaN]
negLog = NaN
sigma = [NaN, NaN]
negLog = NaN
sigma = [1.0248237372088926, 0.01318516660603881]
negLog = -1786.9679007265013
sigma = [NaN, NaN]
negLog = NaN
sigma = [1.04211772992

(-1.968469929933476e6, [1.0248268796675095, 1.1625301820233277e-5], :XTOL_REACHED)

In [54]:
if verbose println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $(opt.numevals) iterations (returned $(ret))") ; end

got -1.96846992993e6 at [1.02483, 1.0e-5] after 61 iterations (returned XTOL_REACHED)
